# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
# from pymongo import MongoClient
# client = MongoClient("")

#esto hay que copiarlo y pegarlo cambiando el dbname en cada data que vayamos a trbaaajr. lo de mas es igual

from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
#ascending and descending hay q importarlos a parte. no vienen con mongoclient

dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL)
db = client.get_database()



mongodb://localhost/companies


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [2]:
#db.companies find nos crea una lista con los parametros que le indicamos. 
#como es una lista le decimos que imprima iterando.
cursor = db.companies.find({'name':'Babelgum'}, {"name":1})
for e in cursor:
    print(e['name'])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [5]:
cursor = db.companies.find({'number_of_employees':{'$gt':5000}}).sort([('number_of_employees', DESCENDING)]).limit(3)
for e in cursor:
    print(e['name'], e['number_of_employees'])
#de ahora en adelante limitare a 3 para que no sea un tocho


Siemens 405000
IBM 388000
Toyota 320000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [6]:
years = list(range(2000, 2006))
cursor = db.companies.find({'founded_year':{'$in':years}}, {'name': 1, 'founded_year': 1}).limit(3)
for e in cursor:
    print(e['name'], e['founded_year'])

Wetpaint 2005
Zoho 2005
Omnidrive 2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [13]:
resp = db.companies.find({'$and':[{'ipo.valuation_amount':{'$gt': 100000000}},
                                    {'founded_year':{'$lt':2010}}]}, 
                           {'name':1,'ipo':1})
for i,e in enumerate(resp):
    if i == 3: break
    print(e['name'], e['ipo'])
#OJO Respuesta de David. Repasar el break y los putos diccionarios. 

Twitter {'valuation_amount': 18100000000, 'valuation_currency_code': 'USD', 'pub_year': 2013, 'pub_month': 11, 'pub_day': 7, 'stock_symbol': 'NYSE:TWTR'}
Facebook {'valuation_amount': 104000000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 5, 'pub_day': 18, 'stock_symbol': 'NASDAQ:FB'}
Yelp {'valuation_amount': 1300000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 3, 'pub_day': 2, 'stock_symbol': 'NYSE:YELP'}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [14]:
resp = db.companies.find({'$and':[{'number_of_employees':{'$lt':1000}}, {'founded_year':{'$lt':2005}}]}).sort([('number_of_employees', DESCENDING)]).limit(10)

for i,e in enumerate(resp):
    if i == 5: break
    print(e['name'], e['number_of_employees'], e['founded_year'])

Infinera Corporation 974 2000
NorthPoint Communications Group 948 1997
888 Holdings 931 1997
Forrester Research 903 1983
SonicWALL 900 1991


### 6. All the companies that don't include the `partners` field.

In [15]:
resp = db.companies.find({'partners':{'$exists':'false'}})

for i,e in enumerate(resp):
    if i == 5: break
    print(e['name'])

AdventNet
Wetpaint
Zoho
Postini
Omnidrive


### 7. All the companies that have a null type of value on the `category_code` field.

In [16]:
resp = db.companies.find({'category_code': None})

for i,e in enumerate(resp):
    if i == 5: break
    print(e['name'])



Collective
Snimmer
KoolIM
Level9 Media
VidKing


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [11]:
resp = db.companies.find({'$and':[{'number_of_employees':{'$gt':100}},{'number_of_employees':{'$lt':1000}}]},
                          {'name':1, 'number_of_employees':1})

for i,e in enumerate(resp):
    if i == 5: break
    print(e['name'], e['number_of_employees'])



AdventNet 600
AddThis 120
OpenX 305
LifeLock 644
Jajah 110


### 9. Order all the companies by their IPO price in a descending order.

In [26]:
resp = db.companies.find({'ipo.valuation_amount':{'$exists':'true'}}).sort([('ipo.valuation_amount', DESCENDING)])

for i,e in enumerate(resp):
    if i == 3: break
    print(e['name'], e['ipo']['valuation_amount'])


GREE 108960000000
Facebook 104000000000
Amazon 100000000000


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [25]:
resp = db.companies.find({'number_of_employees':{'$gt':1}}).sort([('number_of_employees', DESCENDING)]).limit(10)



    #sigo sin entender el puto break aqui pero funciona Es lo mismo que limitarlo? 
    #OJOO algo estaba haciendo mal pq si que funciona si el break ahora. Estaba limitand mal. Es otra manera. 

Siemens 405000
IBM 388000
Toyota 320000
PayPal 300000
Nippon Telegraph and Telephone Corporation 227000
Samsung Electronics 221726
Accenture 205000
Tata Consultancy Services 200300
Flextronics International 200000
Safeway 186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [27]:
resp = db.companies.find({'founded_month':{'$gt':5}}).limit(3)

for i,e in enumerate(resp):
    print(e['name'], e['founded_month'])



Wetpaint 10
Zoho 9
Postini 6


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [30]:
resp = db.companies.find({'$and':[{'acquisition.price_amount':{'$gt':10000000}},{'founded_year':{'$lt': 2000}}]})

for i,e in enumerate(resp):
    if i == 3:break
    print(e['name'], e['acquisition']['price_amount'], e['founded_year'])
    
#vuelvo a utilizar el break para no limitar en la propia funcion, pero funciona exactamente igual. 

Postini 625000000 1999
SideStep 180000000 1999
Recipezaar 25000000 1999


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [33]:
resp = db.companies.find({'acquisition.acquired_year':{'$gt':2010}}, {'name':1, 'acquisition':1}).sort([('acquisition.price_amount', DESCENDING)])

for i,e in enumerate(resp):
    if i == 1: break
    print(e['name'],e['acquisition'])

T-Mobile {'price_amount': 39000000000, 'price_currency_code': 'USD', 'term_code': None, 'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/', 'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion', 'acquired_year': 2011, 'acquired_month': 3, 'acquired_day': 20, 'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}


## 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [ ]:
###

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [34]:
resp = db.companies.find({'founded_day':{'$lte':7}}).sort([('acquisition.price_amount', DESCENDING)]).limit(3)

for i,e in enumerate(resp):
   
    print(e['name'], e['founded_day'])
    
#limito a 3

Netscape 4
PayPal 1
Zappos 1


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [36]:


resp = db.companies.find({'$and':[{'category_code':{'$eq':'web'}},{'number_of_employees':{'$gt':4000}}]}).sort([('number_of_employees', ASCENDING)])

for i,e in enumerate(resp):
    if i == 3: break
    print(e['name'], e['category_code'], e['number_of_employees'])



Expedia web 4400
AOL web 8000
Webkinz web 8657


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code